# This notebook contains the experiments on Adult Census dataset with LionForests

In [1]:
import sys
cpath = !pwd
sys.path.append('/usr/src/app/algorithms/')
sys.path.append('/usr/src/app/')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts
from algorithms.anchor.anchor_tabular import AnchorTabularExplainer

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.metrics import f1_score, precision_score

Feature Engineering from:
https://github.com/pooja2512/Adult-Census-Income/blob/master/Adult%20Census%20Income.ipynb. So run and skip the next code block

In [3]:
adult = Dataset()
X, y, feature_names, class_names, categorical_features = adult.load_adult()
y = np.array(y)

In [4]:
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn

In [13]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, random_state=10):
    parameters = [{
        'max_depth': [5],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf' : [2],
        'n_estimators': [500]
    }]
    scaler = MinMaxScaler(feature_range=(-1,1))
    lf = LionForests(None, False, scaler, feature_names, class_names)#, categorical_features)
    lf.fit(X_train, y_train, params=parameters)
    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task, random_state=10):

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance, reduction=True, ar_algorithm='fpgrowth', cl_algorithm='kmedoids', method='123', instance_random_state=random_state)[-1]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule
        print('    LF Ready')
        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task, random_state)
        print('    GT Ready')
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150, random_state)
        print('    LT Ready')
        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=10, qitr=0, tol=1e-6, restart=2, verbose=0,seed = random_state)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            length, nodes = mdl.find_rule_length(instances[0])
            max_len = len(length)
            comp = {1:'>',0:'<='}
            rules = {}
            for f in feature_names:
                rules[f] = []
            counter = 0
            for rule, node in list(zip(mdl.rule_,nodes)):
                for conj in range(len(node)):
                    if node[conj] and counter<=max_len-1:
                        rules[feature_names[rule[conj][0]-1]].append([comp[rule[conj][1]],rule[conj][2]])
                        counter += 1
            new_rules = {}
            for k, v in rules.items():
                if len(v) == 1:
                    new_rules[k] = v
                else:
                    mmin = None
                    mmax = None
                    for value in v:
                        if value[0]=='<=':
                            if mmin is None or mmin > value[1]:
                                mmin = value[1]
                        if value[0]=='>':
                            if mmax is None or mmax < value[1]:
                                mmax = value[1]
                    if mmin is not None:
                        new_rules[k] = [['<=', mmin]]
                    if mmax is not None:
                        new_rules[k] = [['>', mmax]]
            te = time.time()
            #print(mdl)
            return length, cover, def_predictions, te-ts, new_rules
        print('    Defrag Ready')

        #Anchors =======================================================================================
        explainer = AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95, random_state=random_state)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0
        print('    Anchors Ready')
        
        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_frame = pd.DataFrame(np.hstack((train,y_train.reshape(len(y_train),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)

        temp_frame = temp_frame.replace({"class": {1: 2}})
        temp_frame = temp_frame.replace({"class": {0: 1}})

        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'adult', random_state=random_state)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=random_state,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time() - chts)/len(test)
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte
        print('    CHIRPS Ready')
        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
        #return {'lf':lf_rule}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification', random_state)
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        #covered = False
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        #covered = False
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        #covered = False
                        return 0
        
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    rules = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    
    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)    
    ktime = time.time()
    for test_ind in range(len(test)):
        if test_ind % 100 ==0:
            print(round(test_ind/(len(test))*100,2),'in:', time.time()-ktime)
            ktime = time.time()
            import pickle
            with open(str(random_state)+'parrot.pkl', 'wb') as f:
                pickle.dump([full_coverage, rule_length, f_precision, time_response], f)
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(test_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules['ch'].append(rule)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te, new_rules = method(np.concatenate((x_test_temp[test_ind:test_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
                rules['df'].append(new_rules)
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
                rules['lf'].append(rule)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[test_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[test_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) >= 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules[name].append(rule)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response, rules

In [15]:
total_results = []
test_size_2 = []
for rand in [7, 10, 77]:
    total_results2 = []
    kf = KFold(n_splits=2)
    folds = 0
    test_size = []
    for train_index, test_index in kf.split(X):
        if folds == 0:
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]
            test_size.append(len(X_test[:500]))
            print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test[:500])))    
            results = measure(X_train, X_test[:500], y_train, y_test[:500], feature_names, class_names, random_state=rand)
            total_results2.append(results)
        folds=folds+1
    test_size_2.append(test_size)
    total_results.append(total_results2)

# of Fold: 1, size of test: 500
    LF Ready
    GT Ready
    LT Ready
    Defrag Ready
    Anchors Ready
len self.path_detail
500
Working on CHIRPS for instance 0 of 500
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 51 patterns from 500 trees for batch_idx 0
start score sort for batch_idx 0 (51) patterns
start merge rule for batch_idx 0 (51) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85837)]
0.9967945941263103 0.5069694192412437 0.13572306390373542 0.20486627043090638
merge complete for batch_idx 0 (51) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 51 patterns from 439 trees for batch_idx 1
start score sort for batch_idx 1 (51) patterns
start merge rule for batch_idx 1 (51) patterns
[('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('occupation_Exec-managerial', False, -0.125), ('education_Elementary-school', True,

found 52 patterns from 500 trees for batch_idx 16
start score sort for batch_idx 16 (52) patterns
start merge rule for batch_idx 16 (52) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85837)]
0.9967945941263103 0.5069694192412437 0.23749612648220048 0.3664672495781563
merge complete for batch_idx 16 (52) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
found 58 patterns from 500 trees for batch_idx 17
start score sort for batch_idx 17 (58) patterns
start merge rule for batch_idx 17 (58) patterns
[('capital-gain', True, -0.898), ('education_HS-grad', False, -0.125), ('capital-loss', True, -0.05167)]
0.9997781475318913 0.3990525104046755 0.11683699876412831 0.18536585365853656
merge complete for batch_idx 17 (58) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
found 50 patterns from 500 trees for batch_idx 18
start 

found 56 patterns from 500 trees for batch_idx 33
start score sort for batch_idx 33 (56) patterns
start merge rule for batch_idx 33 (56) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85837)]
0.9967945941263103 0.5069694192412437 0.23598117296511967 0.3354978354978355
merge complete for batch_idx 33 (56) patterns
start get explainer for batch_idx 33
as_chirps for batch_idx 34
start mining for batch_idx 34 with support = 0.1
found 53 patterns from 500 trees for batch_idx 34
start score sort for batch_idx 34 (53) patterns
start merge rule for batch_idx 34 (53) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85838)]
0.9967945941263103 0.5069694192412437 0.23021389045751442 0.35555555555555574
merge complete for batch_idx 34 (53) patterns
start get explainer for batch_idx 34
Working on CHIRPS for instance 35 of 500
as_chirps for batch_idx 35
start mining for batch_idx 35 with support = 0.1
found 55 patterns from 500 trees for batch_idx

found 47 patterns from 500 trees for batch_idx 50
start score sort for batch_idx 50 (47) patterns
start merge rule for batch_idx 50 (47) patterns
[('marital-status_Married', True, 0.0), ('capital-loss', True, 0.23529), ('capital-gain', True, -0.85812)]
0.9998259355961706 0.5086336668732844 0.28695247641949867 0.4142084133356607
merge complete for batch_idx 50 (47) patterns
start get explainer for batch_idx 50
as_chirps for batch_idx 51
start mining for batch_idx 51 with support = 0.1
found 77 patterns from 180 trees for batch_idx 51
start score sort for batch_idx 51 (77) patterns
start merge rule for batch_idx 51 (77) patterns
[('capital-gain', True, -0.898), ('occupation_Exec-managerial', True, 0.0), ('occupation_Prof-specialty', True, 0.0), ('education_Bachelors', True, 0.0), ('capital-loss', True, -0.05368)]
0.9932484703565652 0.616191091548772 0.06656952075365445 0.06441223832528184
merge complete for batch_idx 51 (77) patterns
start get explainer for batch_idx 51
as_chirps for bat

found 54 patterns from 382 trees for batch_idx 66
start score sort for batch_idx 66 (54) patterns
start merge rule for batch_idx 66 (54) patterns
[('education_Elementary-school', True, 0.0), ('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('education_Bachelors', False, -0.125)]
0.9549323985978968 0.08790017695037598 0.28639503693300883 0.2900375257232779
merge complete for batch_idx 66 (54) patterns
start get explainer for batch_idx 66
as_chirps for batch_idx 67
start mining for batch_idx 67 with support = 0.1
found 53 patterns from 500 trees for batch_idx 67
start score sort for batch_idx 67 (53) patterns
start merge rule for batch_idx 67 (53) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85838)]
0.9967945941263103 0.5069694192412437 0.2302139145241116 0.35555555555555574
merge complete for batch_idx 67 (53) patterns
start get explainer for batch_idx 67
as_chirps for batch_idx 68
start mining for batch_idx 68 with support = 0.1

...

start merge rule for batch_idx 422 (77) patterns
[('capital-gain', True, -0.898), ('occupation_Exec-managerial', True, 0.0), ('occupation_Prof-specialty', True, 0.0), ('education_Bachelors', True, 0.0), ('capital-loss', True, -0.0536)]
0.9932484703565652 0.616191091548772 0.06663174887601568 0.06451612903225801
merge complete for batch_idx 422 (77) patterns
start get explainer for batch_idx 422
as_chirps for batch_idx 423
start mining for batch_idx 423 with support = 0.1
found 58 patterns from 500 trees for batch_idx 423
start score sort for batch_idx 423 (58) patterns
start merge rule for batch_idx 423 (58) patterns
[('capital-gain', True, -0.898), ('education_HS-grad', False, -0.125), ('capital-loss', True, -0.05135)]
0.9997781475318913 0.3990525104046755 0.1109678812315793 0.1802658721063488
merge complete for batch_idx 423 (58) patterns
start get explainer for batch_idx 423
as_chirps for batch_idx 424
start mining for batch_idx 424 with support = 0.1
found 49 patterns from 500 tree

start merge rule for batch_idx 438 (49) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85837)]
0.9967945941263103 0.5069694192412437 0.14109646864481956 0.20324304403906385
merge complete for batch_idx 438 (49) patterns
start get explainer for batch_idx 438
as_chirps for batch_idx 439
start mining for batch_idx 439 with support = 0.1
found 53 patterns from 500 trees for batch_idx 439
start score sort for batch_idx 439 (53) patterns
start merge rule for batch_idx 439 (53) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85838)]
0.9967945941263103 0.5069694192412437 0.22992572562616387 0.35509293680297416
merge complete for batch_idx 439 (53) patterns
start get explainer for batch_idx 439
Working on CHIRPS for instance 440 of 500
as_chirps for batch_idx 440
start mining for batch_idx 440 with support = 0.1
found 53 patterns from 500 trees for batch_idx 440
start score sort for batch_idx 440 (53) patterns
start merge rule for batch_idx

found 52 patterns from 500 trees for batch_idx 455
start score sort for batch_idx 455 (52) patterns
start merge rule for batch_idx 455 (52) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85837)]
0.9967945941263103 0.5069694192412437 0.23285926020915715 0.36783904162187064
merge complete for batch_idx 455 (52) patterns
start get explainer for batch_idx 455
as_chirps for batch_idx 456
start mining for batch_idx 456 with support = 0.1
found 57 patterns from 500 trees for batch_idx 456
start score sort for batch_idx 456 (57) patterns
start merge rule for batch_idx 456 (57) patterns
[('marital-status_Married', True, 0.0), ('capital-gain', True, -0.85837)]
0.9967945941263103 0.5069694192412437 0.24417451655375397 0.3428819444444444
merge complete for batch_idx 456 (57) patterns
start get explainer for batch_idx 456
as_chirps for batch_idx 457
start mining for batch_idx 457 with support = 0.1
found 53 patterns from 500 trees for batch_idx 457
start score sort for b

found 51 patterns from 399 trees for batch_idx 472
start score sort for batch_idx 472 (51) patterns
start merge rule for batch_idx 472 (51) patterns
[('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('occupation_Prof-specialty', False, -0.125), ('education_Elementary-school', True, 0.0)]
0.9857142857142858 0.06492787771746582 0.24937325993511028 0.21044285876090157
merge complete for batch_idx 472 (51) patterns
start get explainer for batch_idx 472
as_chirps for batch_idx 473
start mining for batch_idx 473 with support = 0.1
found 55 patterns from 500 trees for batch_idx 473
start score sort for batch_idx 473 (55) patterns
start merge rule for batch_idx 473 (55) patterns
[('marital-status_Married', True, 0.0), ('capital-loss', True, 0.23691), ('capital-gain', True, -0.85823)]
0.9998259355961706 0.5086336668732844 0.23096918709233397 0.3725243634077333
merge complete for batch_idx 473 (55) patterns
start get explainer for batch_idx 473
as_chirps for batch_idx

found 50 patterns from 500 trees for batch_idx 488
start score sort for batch_idx 488 (50) patterns
start merge rule for batch_idx 488 (50) patterns
[('capital-gain', True, -0.898), ('education_HS-grad', False, -0.125), ('capital-loss', True, -0.05186)]
0.9997781475318913 0.3990525104046755 0.13661592205097017 0.20306995470558642
merge complete for batch_idx 488 (50) patterns
start get explainer for batch_idx 488
as_chirps for batch_idx 489
start mining for batch_idx 489 with support = 0.1
found 55 patterns from 420 trees for batch_idx 489
start score sort for batch_idx 489 (55) patterns
start merge rule for batch_idx 489 (55) patterns
[('education_HS-grad', True, 0.0), ('marital-status_Married', False, -0.125), ('education_Elementary-school', True, 0.0), ('occupation_Exec-managerial', False, -0.125)]
0.9745762711864406 0.06248770852223913 0.279354296389589 0.23325857097084243
merge complete for batch_idx 489 (55) patterns
start get explainer for batch_idx 489
Working on CHIRPS for ins

In [16]:
from utilities.lionforests_utility import path_similarity
rule_variance = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}


kf = KFold(n_splits=2)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    if folds == 0:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        min_max_feature_values = {}
        for i in range(len(feature_names)):
                min_max_feature_values[feature_names[i]] = [min(X_train[:, i]), max(X_train[:, i])] 
        for name in ['gs', 'ls', 'an', 'lf', 'df', 'ch']:
            for k in range(len(total_results[0][folds][-1][name])):
                r1 = total_results[0][0][-1][name][k]
                r2 = total_results[1][0][-1][name][k]
                r3 = total_results[2][0][-1][name][k]
                #try:
                rule_variance[name].append((path_similarity(r1, r2, feature_names, min_max_feature_values)+
                                               path_similarity(r1, r3, feature_names, min_max_feature_values)+
                                               path_similarity(r2, r3, feature_names, min_max_feature_values))/3)
                #except TypeError:
                #    print(name, r1,r2,r3)
    folds = folds + 1

In [17]:
f_full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
f_f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
for b in range(3):
    full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    k = 0
    for i in total_results[b]:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/test_size_2[b][k])
            rule_length[name].append(i[2][name]/test_size_2[b][k])
            f_precision[name].append(np.array(i[3][name]).mean())
            f_time[name].append(np.array(i[4][name]).mean())
        k = + 1
    for name, method in i[0].items():
        f_full_coverage[name].append(np.array(full_coverage[name]).mean())
        f_rule_length[name].append(np.array(rule_length[name]).mean())
        f_f_precision[name].append(np.array(f_precision[name]).mean())
        f_f_time[name].append(np.array(f_time[name]).mean())
for name, method in i[0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()))

gs | 0.3096  0.000 | 4.2833 0.022 | 1.0000  0.000 | 0.0004  0.000 | 0.9677  0.021
ls | 0.6768  0.006 | 1.6160 0.029 | 0.8042  0.001 | 3.5882  0.031 | 0.9778  0.026
an | 0.3204  0.009 | 2.7080 0.214 | 0.7972  0.002 | 26.6188  1.102 | 0.9718  0.077
lf | 0.0206  0.000 | 9.8940 0.078 | 1.0000  0.000 | 1.4780  0.019 | 0.9880  0.021
df | 0.9993  0.001 | 6.0073 2.336 | 0.8507  0.006 | 0.4941  0.041 | 0.9348  0.008
ch | 0.4204  0.001 | 2.7880 0.002 | 0.9900  0.000 | 6.8563  0.049 | 0.9742  0.012


In [18]:
import csv  

with open('adult.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    for name, method in i[0].items():
        writer.writerow([name, np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()])